# Folium

Folium is a interactive map plotting library that is based on top of Leaflet.js, a popular Javascript plotting library. Since it's based on Javascript, we can easily export our plots into HTML and use them inside a website.


Folium is a Python library for creating interactive maps visualizations. It provides a high-level interface for drawing maps using the Leaflet JavaScript library. With Folium, you can easily create maps of different styles, add markers, popups, and other features, and control the appearance of your maps.

Folium is particularly useful for data visualization and exploration, as it allows you to quickly create maps to display and explore your data. You can use Folium to create choropleth maps, heat maps, and other types of visualizations, and easily integrate them into your workflow. Additionally, Folium makes it easy to export your maps to various formats, including HTML, images, and standalone interactive maps.

Overall, Folium is a powerful tool for data exploration and visualization, and is widely used in the fields of geography, GIS, data science, and more.


# Map Formats

If we want to draw something on a map like the border of a country, or district, we need a file to specify the polygon boundaries. These come in a variety of formats but some common ones are:

* Shape files
* Geo JSON
* TOPO JSON

Shape files are the oldest format and as such are normally easier to find. Shape files for many countries can be found [here](https://gadm.org/index.html). It can be slightly trickier to find GEO JSON or TOPO JSON files online. 

However, many of the interactive plotting libraries we'd like to use are Javascript-based and hence normally use GEO or TOPO JSON. Thankfully, we can use PyShp to convert shape files to GEO or TOPO so we can use them with folium.  

TOPO JSON is more compact than GEO JSON, as it limits redundancy by representing shared geometries together (imagine two countries with a shared border). However if file size isn't a consideration, GEO JSON is simpler and hence preferred.



In [ ]:
!pip install pyshp
import shapefile
from json import dumps
import json

In [ ]:
infile = "data/shapefiles/gadm36_HKG_1.shp"
outfile  = "data/hk_geo.json"

def shape2json(infile,outfile):
    # Read the shapefile
    reader = shapefile.Reader(infile)
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]
    buffer = []
    for sr in reader.shapeRecords():
        atr = dict(zip(field_names, sr.record))
        geom = sr.shape.__geo_interface__
        buffer.append(dict(type="Feature", geometry=geom, properties=atr)) 

    # write the GeoJSON file
    geojson = open(outfile, "w")
    geojson.write(dumps({"type": "FeatureCollection", "features": buffer}, indent=2) + "\n")
    geojson.close()

shape2json(infile,outfile)

Let's look a little more at the structure of the geo json file.

In [ ]:
geojson = json.load(open("data/hk_geo.json"))
geojson.keys()

The type is a feature collection which contains a list of 18 features. 

In [ ]:
geojson["type"] #

In [ ]:
len(geojson["features"]) # list of length 18 , one for each district

Let's look at a single feature in more detail.

In [ ]:
feature  = geojson["features"][0]

Each feature represents a single district and has 3 keys.

In [ ]:
feature.keys() 

It has a type of feature.

In [ ]:
feature["type"]

A geometry is like a dictionary, which specifies the type of shape and a list or coordinates.

In [ ]:
feature["geometry"].keys()

In [ ]:
feature["geometry"]["type"]

The coordinates are within a nested list.

In [ ]:
feature["geometry"]["coordinates"][0][0][:5]

A properties key which contains some meta data.

In [ ]:
geojson["features"][0]["properties"]

We could get the name of each district:

In [ ]:
[ each["properties"]["NAME_1"] for each in geojson["features"] ]

# Plotting

Now that we've got our geo json,  we can plot it on a map.

In [ ]:
!pip install folium
import folium

In [ ]:
lon, lat = 22.3964, 114.1095
hk_map = folium.Map(location=[lon, lat],
                   tiles="OpenStreetMap", zoom_start=11)

#add area overlays
folium.GeoJson(geojson).add_to(hk_map)
hk_map

We'll add a style function to colour each distict blue except Kowloon City which we'll colour red. The style function will get called for each feature in the feature collection.

In [ ]:
def style_function(x):
    d = {
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
    
    if x['properties']['NAME_1'] == "Kowloon City":
        d["fillColor"] ="red"
    else:
        d["fillColor"] ="blue"
        
    return d

We can change the map's appearance by changing the titles keyword.

In [ ]:

hk_map = folium.Map(location=[lon, lat],
                   tiles='Stamen Terrain', zoom_start=11)

#add area overlays
folium.GeoJson(geojson,style_function=style_function).add_to(hk_map)
hk_map

For more example of maps using Follium see [here](http://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/)